In [1]:
import csbgnpy.pd.io.cd
import csbgnpy.pd.io.sbgnml
import sbgn2an.utils
import sbgn2an.ini

## Merged map

From [1 - Merge CYCLE and CLOCK](1%20-%20Merge%20CYCLE%20and%20CLOCK.ipynb)

In [2]:
merge = csbgnpy.pd.io.sbgnml.read("generated/MERGE.sbgnml")

## Precursor states

We compute all precursor states that respect the constraints given in the paper, and that contains the pRB:E2F1:DP2 complex.

In [3]:
inis = sbgn2an.ini.get_ini(merge, in_ini=[merge.get_entity("Complex([SubMacromolecule([][@|@|@|@]pRB*)|SubMacromolecule([][@]DP1*)|SubMacromolecule([][@Ser364|@Ser31|@Ser403|@Lys|@]E2F1)][][]pRB*/E2F1/DP1*#Compartment(nucleoplasm))", by_string=True)])
print(len(inis))

24


We obtain 24 possible precursor states.
We check the difference between each initial state.
For this we first compute the intersection of all precursor states.

In [4]:
inter = set([e.id for e in inis[0]])
for ini in inis[1:]:
    inter = inter.intersection(set([e.id for e in ini]))

In [5]:
print([len(ini) for ini in inis])
print(len(inter))

[106, 106, 106, 106, 106, 106, 106, 106, 106, 106, 106, 106, 106, 106, 106, 106, 106, 106, 106, 106, 106, 106, 106, 106]
102


Each of the 24 precursor states is constituted of 106 entities, 102 of which are common to all precursor states.
Hence we check the nature of the entities that do not belong to all initial states.

In [6]:
tot_diff = set([])
for ini in inis:
    ini = set([e.id for e in ini])
    diff = ini.difference(inter)
    tot_diff = tot_diff.union(diff)
    
print([str(merge.get_entity(e, by_id=True)) for e in tot_diff])


['Macromolecule([][]DP2*#Compartment([]nucleoplasm))', 'SimpleChemical([][]NAM#Compartment([]nucleoplasm))', 'SimpleChemical([][]NMN#Compartment([]nucleoplasm))', 'Macromolecule([][]E2F4#Compartment([]cytoplasm))', 'Macromolecule([][@|@|@Ser15|@Ser20]TP53#Compartment([]nucleoplasm))', 'Macromolecule([][]E2F4#Compartment([]nucleoplasm))', 'SimpleChemical([][]NAD+#Compartment([]nucleoplasm))', 'Macromolecule([][]DP2*#Compartment([]cytoplasm))', 'Macromolecule([][@|@|@Ser15|@Ser20]TP53#Compartment([]cytoplasm))']


For each initial state, the 4 entities that do not belong to all initial states are to be chosen as follows (2 x 2 x 2 x 3 = 24 possibilities): 

* TP53: "Macromolecule([][@|@|@Ser15|@Ser20]TP53#Compartment([]cytoplasm))" or "Macromolecule([][@|@|@Ser15|@Ser20]TP53#Compartment([]nucleoplasm))"
* E2F4: "Macromolecule([][]E2F4#Compartment([]cytoplasm))" or "Macromolecule([][]E2F4#Compartment([]nucleoplasm))"
* DP2: "Macromolecule([][]DP2\*#Compartment([]cytoplasm))" or "Macromolecule([][]DP2*#Compartment([]nucleoplasm))"
* NAD+/NAM/NMN: "SimpleChemical([][]NAD+#Compartment([]nucleoplasm))" or "SimpleChemical([][]NAM#Compartment([]nucleoplasm))" or "SimpleChemical([][]NMN#Compartment([]nucleoplasm))"

For p53, E2F4, DP2, the entities have two localizations (cytoplasm and nucleoplasm) and cycling translocation processes from one localization to the other.
For the NAD+/NAM/NMN entities, there is a cycle producing the three entities (NAD+->NAM->NMN->NAD+).

We consider all initial states. Each initial state is encoded following the choice made for each of the four entities:

In [7]:
nad = ["SimpleChemical([][]NAD+#Compartment([]nucleoplasm))", "SimpleChemical([][]NAM#Compartment([]nucleoplasm))", "SimpleChemical([][]NMN#Compartment([]nucleoplasm))"] # code: [1XXX, 2XXX, 3XXX]
e2f4 = ["Macromolecule([][]E2F4#Compartment([]cytoplasm))", "Macromolecule([][]E2F4#Compartment([]nucleoplasm))"] # code: [X1XX, X2XX]
dp2 = ["Macromolecule([][]DP2*#Compartment([]cytoplasm))", "Macromolecule([][]DP2*#Compartment([]nucleoplasm))"] # code: [XX1X, XX2X]
p53 = ["Macromolecule([][@|@|@Ser15|@Ser20]TP53#Compartment([]cytoplasm))", "Macromolecule([][@|@|@Ser15|@Ser20]TP53#Compartment([]nucleoplasm))"] # code: [XXX1, XXX2]
choices = [nad, e2f4, dp2, p53]

dinis = {}

for ini in inis:
    sini = [str(e) for e in ini]
    code = []
    for i in range(len(choices)):
        for j in range(len(choices[i])):
            if choices[i][j] in sini:
                code.append(j + 1)
    assert len(code) == 4
    dinis["".join([str(e) for e in code])] = ini

inis = dinis
%store inis

Stored 'inis' (dict)


### Inputs of CLOCK in the precursor states

In [8]:
inputs = ["UnspecifiedEntity(Heat#Compartment([]extracellular))", "SimpleChemical([][]GC#Compartment([]extracellular))"]

In [9]:
for code, ini in inis.items():
    print(code, "{}/{}".format(len([e for e in inputs if merge.get_entity(e, by_string=True) in ini]), len(inputs)))

3121 2/2
3221 2/2
1121 2/2
1221 2/2
3111 2/2
3211 2/2
1111 2/2
1211 2/2
3122 2/2
1122 2/2
3222 2/2
1222 2/2
3112 2/2
1112 2/2
3212 2/2
1212 2/2
2121 2/2
2221 2/2
2122 2/2
2222 2/2
2111 2/2
2112 2/2
2211 2/2
2212 2/2
